**TODO**
* Create parameter dictionary
* `blockTools` wrapper
  * Transform raw output
* Write cross-validation process

### Import modules

In [14]:
import numpy as np
import pandas as pd

import sklearn.datasets as datasets
import sklearn.linear_model as linear_model

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

### Declare parameters

In [2]:
Y_TYPE = 'binary'

### Define functions

In [24]:
def simulate_data(y_type):
    '''Create synthetic data.'''
    if y_type == 'binary':
        X, y = datasets.make_classification(n_samples=100, n_features=20, n_informative=2,
                                            n_redundant=2, n_repeated=0, n_classes=2,
                                            n_clusters_per_class=2, weights=None, flip_y=0.01,
                                            class_sep=1.0, hypercube=True, shift=0.0, scale=1.0,
                                            shuffle=True, random_state=None)
    elif y_type == 'multi_class':
        X, y = datasets.make_multilabel_classification(n_samples=100, n_features=20, n_classes=5,
                                                       n_labels=2, length=50,
                                                       allow_unlabeled=True, sparse=False,
                                                       return_indicator='dense',
                                                       return_distributions=False,
                                                       random_state=None)
    elif y_type == 'continuous':
        X, y = datasets.make_regression(n_samples=100, n_features=100, n_informative=10,
                                        n_targets=1, bias=0.0, effective_rank=None,
                                        tail_strength=0.5, noise=0.0, shuffle=True, coef=False,
                                        random_state=None)
    return X, y

def block(X):
    '''Assign each sample to a block.'''
    # Activate pandas conversion support
    pandas2ri.activate()
    # Import blockTools
    block = importr('blockTools')
    # Convert X to pandas DataFrame 
    df = pd.DataFrame(X, columns=['v{}'.format(i + 1) for i in xrange(X.shape[1])]).reset_index()
    # Perform blocking
    out = block.block(df, id_vars='index', block_vars='v1', n_tr=10)
    # Extract assignment DataFrame
    return pandas2ri.ri2py_dataframe(out.rx2('blocks').rx2('1'))

### Perform analysis

In [4]:
# Create synthetic data
X, y = simulate_data(Y_TYPE)

# Use blockTools for cross-validation assignment
cval_blocks = block(X)